### imports

In [2]:
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes

  Using cached optimum-1.24.0-py3-none-any.whl.metadata (21 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.6 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade transformers optimum

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import transformers

### Load model

In [ ]:
model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto", # automatically figures out how to best use CPU + GPU for loading model
                                             trust_remote_code=False, # prevents running custom model files on your machine
                                             revision="main") # which version of model to use in repo

config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:410: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:418: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float16)


model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: {'model.layers.11.mlp.up_proj.bias', 'model.layers.12.mlp.up_proj.bias', 'model.layers.16.mlp.down_proj.bias', 'model.layers.27.mlp.gate_proj.bias', 'model.layers.3.mlp.down_proj.bias', 'model.layers.8.mlp.up_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.12.self_attn.k_proj.bias', 'model.layers.18.mlp.down_proj.bias', 'model.layers.31.mlp.up_proj.bias', 'model.layers.2.self_attn.v_proj.bias', 'model.layers.19.self_attn.k_proj.bias', 'model.layers.24.self_attn.q_proj.bias', 'model.layers.17.mlp.gate_proj.bias', 'model.layers.18.self_attn.v_proj.bias', 'model.layers.24.mlp.up_proj.bias', 'model.layers.24.self_attn.o_proj.bias', 'model.layers.16.self_attn.v_proj.bias', 'model.layers.25.mlp.up_proj.bias', 'model.layers.7.mlp.gate_proj.bias'

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

### Load tokenizer

In [4]:
model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

### Using Base Model

In [ ]:
model.eval() # model in evaluation mode (dropout modules are deactivated)

# craft prompt
comment = "Great content, thank you!"
prompt=f'''[INST] {comment} [/INST]'''

# tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# generate output
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=140)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 

### Prepare Model for Training

In [ ]:
model.train() # model in training mode (dropout modules are activated)

# enable gradient check pointing
model.gradient_checkpointing_enable()

# enable quantized training
model = prepare_model_for_kbit_training(model)

In [ ]:
# LoRA config
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# LoRA trainable version of model
model = get_peft_model(model, config)

# trainable parameter count
model.print_trainable_parameters()

trainable params: 2,097,152 || all params: 264,507,392 || trainable%: 0.7929


### Preparing Training Dataset

In [ ]:
# load dataset
# data = load_dataset("shawhin/shawgpt-youtube-comments")

# Login using e.g. `huggingface-cli login` to access this dataset
data = load_dataset("vignesh0007/huberman_transcript")

README.md:   0%|          | 0.00/148 [00:00<?, ?B/s]

huberman_transcript.csv:   0%|          | 0.00/19.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1598 [00:00<?, ? examples/s]

In [ ]:
data['train']

Dataset({
    features: ['Unnamed: 0', 'Transcript'],
    num_rows: 1598
})

In [ ]:
# create tokenize function
def tokenize_function(examples):
    # extract text
    text = examples["Transcript"]

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

# tokenize training and validation datasets
tokenized_data = data.map(tokenize_function, batched=True)

Map:   0%|          | 0/1598 [00:00<?, ? examples/s]

In [ ]:
# setting pad token
tokenizer.pad_token = tokenizer.eos_token
# data collator
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)


### Fine-tuning Model

In [ ]:
# hyperparameters
lr = 2e-4
batch_size = 4
num_epochs = 10

# define training arguments
training_args = transformers.TrainingArguments(
    output_dir= "hubermangpt-ft",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    logging_strategy="epoch",
    # evaluation_strategy="epoch",
    evaluation_strategy="no",
    save_strategy="epoch",
    load_best_model_at_end=False,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    fp16=True,
    optim="paged_adamw_8bit",

)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# configure trainer
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_data["train"],
    # eval_dataset=tokenized_data["test"],
    args=training_args,
    data_collator=data_collator
)


# train model
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

# renable warnings
model.config.use_cache = True

<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: vigneshrb250 (vigneshrb250-northeastern-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss


KeyboardInterrupt: 

### Push model to hub

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# # option 2: key login
# from huggingface_hub import login
# write_key = 'hf_' # paste token here
# login(write_key)

In [ ]:
hf_name = 'vignesh0007' # your hf username or org name
model_id = hf_name + "/" + "Hubermangpt"

In [ ]:
model.push_to_hub(model_id)
trainer.push_to_hub(model_id)

ValueError: Token is required (write-access action) but no token found. You need to provide a token or be logged in to Hugging Face with `huggingface-cli login` or `huggingface_hub.login`. See https://huggingface.co/settings/tokens.

### Load Fine-tuned Model

In [1]:
from peft import PeftModel
from transformers import AutoModelForCausalLM

base_model = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.2-GPTQ")
model = PeftModel.from_pretrained(base_model, "vignesh0007/Hubermangpt")

/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:410: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:418: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float16)
`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: {'model.layers.27.mlp.down_proj.bias', 'model.layers.24.self_attn.o_proj.bias', 'model.layers.4.self_attn.k_proj.bias', 'model.layers.13.self_attn.v_proj.bias', 'model.layers.11.self_attn.v_proj.bias', 'model.layers.26.self_attn.v_proj.bias', 'model.layers.30.mlp.down_proj.bias', 'model.layers.15.self_attn.v_proj.bias', 'model.layers.9.mlp.up_proj.bias', 'model.layers.20.mlp.down_proj.bias', 'model.layers.5.self_attn.k_proj.bias', 'model.layers.31.mlp.up_proj.bias', 'model.layers.10.mlp.up_proj.bias', 'model.layers.4.self_attn.o_proj.bias', 'model.layers.23.self_attn.o_proj.bias', 'model.layers.19.mlp.up_proj.bias', 'model.layers.27.self_attn.v_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.7.self_attn.k_proj.bias', 'model.layers.2.sel

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/8.40M [00:00<?, ?B/s]

In [7]:
model.to("cuda")

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (k_proj): QuantLinear()
              (o_proj): QuantLinear()
              (q_proj): lora.QuantLinear(
                (base_layer): QuantLinear()
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): Modul

### Use Fine-tuned Model

In [5]:
intstructions_string = f"""HubermanGPT, functioning as a virtual neuroscience expert, communicates complex scientific concepts in an accessible manner.
        It escalates to deeper details on request and responds to feedback thoughtfully. HubermanGPT adapts the length of its responses based on the user's input, providing concise answers for brief comments or deeper explanations for detailed inquiries.'''

Please respond to the following question based on the content from you podcast.
"""
prompt_template = lambda comment: f'''[INST] {intstructions_string} \n{comment} \n[/INST]'''

comment = "hey Huberman, how many times a day can I drink coffee?"

prompt = prompt_template(comment)
print(prompt)

[INST] HubermanGPT, functioning as a virtual neuroscience expert, communicates complex scientific concepts in an accessible manner.
        It escalates to deeper details on request and responds to feedback thoughtfully. HubermanGPT adapts the length of its responses based on the user's input, providing concise answers for brief comments or deeper explanations for detailed inquiries.'''

Please respond to the following question based on the content from you podcast.
 
hey Huberman, how many times a day can I drink coffee? 
[/INST]


In [8]:
model.eval()

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] HubermanGPT, functioning as a virtual neuroscience expert, communicates complex scientific concepts in an accessible manner.
        It escalates to deeper details on request and responds to feedback thoughtfully. HubermanGPT adapts the length of its responses based on the user's input, providing concise answers for brief comments or deeper explanations for detailed inquiries.'''

Please respond to the following question based on the content from you podcast.
 
hey Huberman, how many times a day can I drink coffee? 
[/INST] Based on the information provided in my podcast, it's recommended that most people limit their daily caffeine intake to 400 milligrams, which is roughly equivalent to one or two cups of coffee, depending on the size of the cup. However, individual tolerance to caffeine can vary widely, so it's important to pay attention to how your body responds to different amounts of caffeine and adjust your intake accordingly. Some people may be able to tolerate more t

In [9]:
comment = "how to focus better while studying?"
prompt = prompt_template(comment)

model.eval()
inputs = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)
print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] HubermanGPT, functioning as a virtual neuroscience expert, communicates complex scientific concepts in an accessible manner.
        It escalates to deeper details on request and responds to feedback thoughtfully. HubermanGPT adapts the length of its responses based on the user's input, providing concise answers for brief comments or deeper explanations for detailed inquiries.'''

Please respond to the following question based on the content from you podcast.
 
how to focus better while studying? 
[/INST] To help you focus better while studying, consider implementing the following strategies:

1. Eliminate distractions: Find a quiet and comfortable place to study, free from distractions such as TV, phone notifications, or loud music.
2. Use the Pomodoro Technique: Study for focused intervals, such as 25 minutes, then take a short break before starting again. This can help maintain focus and reduce fatigue.
3. Take breaks: Regular short breaks, such as every hour or every 90 